In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pygad

module_path = os.path.abspath(os.path.join('../')) # or the path
sys.path.append(module_path)

import itertools
import pandas as pd
import numpy as np

from engine import Bool
from bnn import ProdTron, SumTron, ProdLayer, SumLayer

In [2]:
# can we learn an xor gate?
def optim_xor(x,w,y):
    h = [ xi^wi for xi, wi in zip(x,w)]
    yh = np.prod(h) # 
    yhd = [yh.data]
    
    # run one iteration of GD
    loss = y^yh # xor between observed and predicted
    loss.backward()

    flips = 0

    # for this specific case, gradient update takes this form
    print("xxxxxxxxxxx\n[BEFORE] Weight_1_data, Weight_1_grad : {} |  Weight_2_data, Weight_2_grad:{}".format(*[(wi.data, wi.grad) for wi in w]))
    for wi in w:
        if wi.grad == wi.data:
            flips += 1
            wi.data = -wi.data
    print("\n [AFTER] Weight_1_data, Weight_1_grad : {} |  Weight_2_data, Weight_2_grad:{}".format(*[(wi.data, wi.grad) for wi in w]))
    # re-eval the gate
    h = [ xi^wi for xi, wi in zip(x,w)]
    yh = np.prod(h) #
    yhd.append(yh.data)

    mistake = 0

    if loss.data<0:
        print('model is CORRECT')    
        if flips>0:
            print('#',flips,'BUT got updated ---')
            print('model after: ',[wi.data for wi in w])
            print("\n[AFTER] Weight_1_data, Weight_1_grad : {} |  Weight_2_data, Weight_2_grad:{}".format(*[(wi.data, wi.grad) for wi in w]))
            mistake += 1
#             print("#########################################")
            print(f"mistake count after model is CORRECT: {mistake}")
        else:
            pass
            print('and NOT updated +++')
        print("xxxxxxxxxxx")
    else:
        print('model is WRONG.') 
        if flips>0:
            print('#',flips,'and got UPDATED +++')
            print("\n[AFTER] Weight_1_data, Weight_1_grad : {} |  Weight_2_data, Weight_2_grad:{}".format(*[(wi.data, wi.grad) for wi in w]))
            print('model after: ',[wi.data for wi in w])
        else:
            mistake +=1
            print("\n[AFTER] Weight_1_data, Weight_1_grad : {} |  Weight_2_data, Weight_2_grad:{}".format(*[(wi.data, wi.grad) for wi in w]))
            print('but NOT UPDATED ---')
            print(f"mistake count after model is WRONG: {mistake}")
        print("xxxxxxxxxxx")
    
    print('pred before: ',yhd[0],'\npred  after: ',yhd[1])
#     print(f"mistake count : {mistake}")
    return w,yhd,flips,mistake

In [2]:
T = [-1,1]
mistakes = 0
for element in itertools.product(T, repeat=5):
    print('\n***\n')
    y = Bool(element[0])
    x = [Bool(element[1]),Bool(element[2])]
    w = [Bool(element[3]),Bool(element[4])]
    print('input: ',x[0].data, x[1].data)
    print('y: ',y.data)
    print('model: ',w[0].data, w[1].data)
#     x_, w_, y_ = x,w,y
    w,yhd,flips,mistake = optim_xor(x,w,y)
    mistakes += mistake
print('total mistakes: ', mistakes, 'out of', np.power(2,5))



***

input:  -1 -1
y:  -1
model:  -1 -1
xxxxxxxxxxx
[BEFORE] Weight_1_data, Weight_1_grad : (-1, 0) |  Weight_2_data, Weight_2_grad:(-1, 0)

 [AFTER] Weight_1_data, Weight_1_grad : (-1, 0) |  Weight_2_data, Weight_2_grad:(-1, 0)
model is CORRECT
and NOT updated +++
xxxxxxxxxxx
pred before:  -1 
pred  after:  -1

***

input:  -1 -1
y:  -1
model:  -1 1
xxxxxxxxxxx
[BEFORE] Weight_1_data, Weight_1_grad : (-1, 1) |  Weight_2_data, Weight_2_grad:(1, 0)

 [AFTER] Weight_1_data, Weight_1_grad : (-1, 1) |  Weight_2_data, Weight_2_grad:(1, 0)
model is CORRECT
and NOT updated +++
xxxxxxxxxxx
pred before:  -1 
pred  after:  -1

***

input:  -1 -1
y:  -1
model:  1 -1
xxxxxxxxxxx
[BEFORE] Weight_1_data, Weight_1_grad : (1, 0) |  Weight_2_data, Weight_2_grad:(-1, 1)

 [AFTER] Weight_1_data, Weight_1_grad : (1, 0) |  Weight_2_data, Weight_2_grad:(-1, 1)
model is CORRECT
and NOT updated +++
xxxxxxxxxxx
pred before:  -1 
pred  after:  -1

***

input:  -1 -1
y:  -1
model:  1 1
xxxxxxxxxxx
[BEFORE] Weig

* Evaulate last weights

In [3]:
# Using last model weights
c=0
failed_tests = []
for element in itertools.product(T, repeat=5):
#     print('\n***\n')
    y = Bool(element[0])
    x = [Bool(element[1]),Bool(element[2])]
    
    yh = np.prod([ xi^wi for xi, wi in zip(x,w)])
    if yh.data == y.data:
        print('-- Passed! --')
        
    else:
        print(x, y)
        failed_tests.append(((x[0].data, x[1].data), y.data))
        c+=1
print("Total fail: ", c)
print("set of cases where it fails :", set(failed_tests))

[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
[data:1, grad:0, data:1, grad:0] data:1, grad:0
[data:1, grad:0, data:1, grad:0] data:1, grad:0
[data:1, grad:0, data:1, grad:0] data:1, grad:0
[data:1, grad:0, data:1, grad:0] dat

* model fails to update its weights at 4 instances, when its predictions are wrong. 

* There has to atleast 1 weight of two assuming gradient value other than 0 per input x and output y, in order to flip weights? 

In [3]:
# Now with different __mul__( ) method

mistakes = 0
for element in itertools.product(T, repeat=5):
    print('\n***\n')
    y = Bool(element[0])
    x = [Bool(element[1]),Bool(element[2])]
    w = [Bool(element[3]),Bool(element[4])]
    print('input: ',x[0].data, x[1].data)
    print('y: ',y.data)
    print('model: ',w[0].data, w[1].data)
#     x_, w_, y_ = x,w,y
    w,yhd,flips,mistake = optim_xor(x,w,y)
    mistakes += mistake
print('total mistakes: ', mistakes, 'out of', np.power(2,5))



***

input:  -1 -1
y:  -1
model:  -1 -1
xxxxxxxxxxx
[BEFORE] Weight_1_data, Weight_1_grad : (-1, -1) |  Weight_2_data, Weight_2_grad:(-1, -1)

 [AFTER] Weight_1_data, Weight_1_grad : (1, -1) |  Weight_2_data, Weight_2_grad:(1, -1)
model is WRONG.
# 2 and got UPDATED +++

[AFTER] Weight_1_data, Weight_1_grad : (1, -1) |  Weight_2_data, Weight_2_grad:(1, -1)
model after:  [1, 1]
xxxxxxxxxxx
pred before:  1 
pred  after:  1

***

input:  -1 -1
y:  -1
model:  -1 1
xxxxxxxxxxx
[BEFORE] Weight_1_data, Weight_1_grad : (-1, 1) |  Weight_2_data, Weight_2_grad:(1, -1)

 [AFTER] Weight_1_data, Weight_1_grad : (-1, 1) |  Weight_2_data, Weight_2_grad:(1, -1)
model is CORRECT
and NOT updated +++
xxxxxxxxxxx
pred before:  -1 
pred  after:  -1

***

input:  -1 -1
y:  -1
model:  1 -1
xxxxxxxxxxx
[BEFORE] Weight_1_data, Weight_1_grad : (1, -1) |  Weight_2_data, Weight_2_grad:(-1, 1)

 [AFTER] Weight_1_data, Weight_1_grad : (1, -1) |  Weight_2_data, Weight_2_grad:(-1, 1)
model is CORRECT
and NOT updated

* Evaulate last weights

In [4]:
# Using last model weights
c=0
failed_tests = []
for element in itertools.product(T, repeat=5):
#     print('\n***\n')
    y = Bool(element[0])
    x = [Bool(element[1]),Bool(element[2])]
    
    yh = np.prod([ xi^wi for xi, wi in zip(x,w)])
    if yh.data == y.data:
        print('-- Passed! --')
        
    else:
        print(x, y)
        failed_tests.append(((x[0].data, x[1].data), y.data))
        c+=1
print("Total fail: ", c)
print("set of cases where it fails :", set(failed_tests))

[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
[data:-1, grad:0, data:-1, grad:0] data:-1, grad:0
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
[data:1, grad:0, data:1, grad:0] data:-1, grad:0
[data:1, grad:0, data:1, grad:0] data:-1, grad:0
[data:1, grad:0, data:1, grad:0] data:-1, grad:0
[data:1, grad:0, data:1, grad:0] data:-1, grad:0
-- Passed! --
-- Passed! --
-- Passed! --
-- Passed! --
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:-1, grad:0, data:1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
[data:1, grad:0, data:-1, grad:0] data:1, grad:0
-- Passed! --
-- Passed! --
-- Passed! -

* set of cases where alternate `__mul__()` fails: `{((-1, -1), -1), ((-1, 1), 1), ((1, -1), 1), ((1, 1), -1)}`

___________________________________________________________

* set of cases where original `__mul__()` fails:  `{((-1, -1), -1), ((-1, 1), 1), ((1, 1), 1), ((1, -1), 1)}`